# Spring Application Deploy Docker Container Linux

### I) Set up ubuntu virtualbox 16.04

#### 1. Install Mysql
```terminal
sudo apt update
sudo apt-get install mysql-server
mysql_secure_installation
```
#### 2. Install Java
Việc cài đặt Java trên môi trường máy ảo này mục đích là để test xem thử khi chuyển đổi hệ điều hành thì lỗi sẽ được kiểm soát trước khi chạy trên môi trường docker.
```terminal
sudo apt update
sudo apt install openjdk-8-jdk # install Java Development Kit
sudo apt install openjdk-8-jre # install Java Runtime Environment
```
#### 3. Install Docker
```terminal
sudo apt-get update
sudo apt-get install apt-transport-https ca-certificates curl gnupg-agent software-properties-common
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
sudo apt-key fingerprint 0EBFCD88
sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu xenial stable"
```
##### Hello

```terminal
sudo apt-get update
sudo apt-get install docker-ce docker-ce-cli containerd.io
```

Lựa chọn version
#### 4. Install Git

### II) Set up dockerfile Spring App

### III) Set up mysql container

Đầu tiên là pull mysql container về trước
```python
docker pull mysql
```
#### 1. Hướng dẫn sử dụng Mysql

##### Ví dụ khởi động 1 mysql 
```python
docker run --name some-mysql -e MYSQL_ROOT_PASSWORD=<password> -d
```
##### Connect to MYSQL from  the MYSQL command line client
Các lệnh dưới đây sẽ start 1 mysql container mẫu khác và chạy lại mysql command line client với mysql container của bạn, nó cho phép bạn thực thi các câu lệnh SQL trong database mẫu: 
```python
docker run -it --network some-network --rm mysql mysql -h<some-mysql> -u<example-user> -p
```
với \<some-mysql\> là tên chính thức cho mysql container của bạn ( kết nối  tới 1 mạng \<some-network\> nào đó trên Docker Network ).
Đĩa này có thể được sử dụng như 1 client đối với non-Docker hoặc remote instances
```python
docker run -it --rm mysql mysql -h<some.mysql.hot> -u<some-mysql-user> -p<some-password>
```
##### Đối với docker stack deploy và docker-compose
ví dụ stack.yml cho mysql:
```yml
# Use root/example as user/password credentials
version: '3.1'

services:
    db:
    image: mysql
    command: --default-authentication-plugin=mysql_native_password
    restart: always
    environment:
        MYSQL_ROOT_PASSWORD: example
        
    adminer:
        image: adminer
        restart: always
        ports:
            - 8080:8080
```
Run: docker stack deploy -c stack.yml ( OR docker-compose -f stack.yml up ), đợi cho đến khi thêm vào hoàn tất, và xem thử kênh http://swarm-ip:8080, http://locahost:8080, hoặc http://host-ip:8080 ( tùy trường hợp ).

##### Container shell truy cập và xem MySQL logs
Lệnh docker exec cho phép bạn có thể run các dòng lệnh bên trong Docker container. Những dòng lệnh sau đây sẽ cho bạn 1 bash shell bên trong mysql container.
```python
docker exec -it <some-mysql> bash
```
Log đã có sẵn bên trong Docker's container log:
```python
docker logs <some-mysql>
```
##### Sử dụng chỉnh sửa cấu hình file MySQL
Cấu hình mặc định cho MySQL có thể tìm thấy trong /etc/mysql/my.cnf, cũng có thể là /etc/mysql/conf.d hay /etc/mysql/mysql.conf.d. Hãy xem lại trong thư mục mysql của bạn.
Nếu muốn /my/custom/config-file.cnf là đường dẫn và tên của cấu hình file của bạn, bạn có thể start mysql container của bạn như sau ():
```python
docker run --name <some-mysql> -v /my/custom:/etc/mysql/conf.d -e MYSQL_ROOT_PASSWORD=<my-secret-pw> -d mysql:tag
```
Điều này sẽ start 1 container mới \<some-mysql\> với điều kiện MySQL instances sử dụng cài đặt khởi động từ /etc/mysql/my.cnf và /etc/mysql/conf.d/config-file.cnf, với cài đặt sau sẽ được ưu tiên.
###### Cấu hình không cần cnf file
Nhiều cách cấu hình có thể bỏ qua như flags cho mysqld. Điều này sẽ cho phép bạn linh hoạt tùy chỉnh container mà không cần tệp cnf. Ví dụ: nếu bạn muốn thay đổi mã hóa và đối chiếu mặc định cho tất cả các bảng để sử dụng UTF-8 (utf8mb4), chỉ cần chạy như sau:
```python
docker run --name some-mysql -e MYSQL_ROOT_PASSWORD=my-secret-pw -d mysql:tag --character-set-server=utf8mb4 --collation-server=utf8mb4_unicode_ci
```
Nếu bạn muốn thấy 1 danh sách hoàn tất các lựa chọn có sẵn, hãy run:
```python
docker run -it --rm mysql:tag --verbose --help
```
##### Biến môi trường
Khi bạn start 1 mysql image, bạn có thể điều chỉnh cấu hình của MySQL instance bằng cách bỏ vào thêm 1 hoặc nhiều biến môi trường khi chạy lệnh docker run. Các lệnh này được biểu diễn giống như cách hoạt động của các lệnh mysql client để cấu hình môi trường với 1 số biến môi trường như:
###### MYSQL_ROOT_PASSWORD

###### MYSQL_DATABASE

###### MYSQL_USER, MYSQL_PASSWORD

###### MYSQL_ALLOW_ROOT_PASSWORD

###### MYSQL_ONETIME_PASSWORD

##### Docker Secrets
Để thay thế việc lộ các thông tin nhạy cảm khi chạy thêm các biến môi trường, \_FILE được thêm vào phía sau danh sách các biến môi trường để thêm tạo các script cho việc load giá trị từ trong file chứa các loại biến môi trường ấy. Ví dụ:
```python
docker run --name <some-mysql> -e MYSQL_ROOT_PASSWORD_FILE=/run/secrets/mysql-root -d mysql:tag
```
Hỗ trợ danh sách các biến môi trường như trên.

#### 2. Initializing a fresh instance